In [47]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer() #Initialize lemmatizer
from nltk.corpus import wordnet
from collections import Counter
import itertools
import ast

>* We are going to use email data from program-l mailing list to build edges and nodes for the graph.
>* https://www.freelists.org/archive/program-l
> * Let's read file using `pd.read_csv` 

In [48]:
data=pd.read_csv('thread_sample.csv')

>* In `data` dataframe, we have six columns, each representing as follows:
    
    * `thread_id` : unique id for each thread
    * `thread_name` : the first subject of the email
    * `body` : the content of the email 
    * `account` : the email account of the sender 
    * `url` : the url of the email
    * `date` : the date of the email 

In [49]:
data.head()

,thread_id,thread_name,body,account,url,date
0,4007,Temperature-monitoring-project-in-C,"[""Hi people! Right now, I'm currently doing a ...",['chojiro1990' 'soronel.haetir' 'chojiro1990' ...,['https://www.freelists.org/post/program-l/Tem...,['2013-06-05T09:50:54.000000000' '2013-06-05T1...
1,7725,subscription,"['I am. I write you from Budapest, Hungary. I ...",['hedvig.jung' 'dzhovani.chemishanov' 'hedvig....,['https://www.freelists.org/post/program-l/sub...,['2021-05-18T14:26:59.000000000' '2021-05-18T1...
2,1162,jaws-remote-desktop,"[""Hi All. I'm possibly needing to use windows ...",['justind' 'david.lant' 'justind' 'travis' 'ju...,['https://www.freelists.org/post/program-l/jaw...,['2005-08-31T08:52:21.000000000' '2005-08-31T0...
3,2300,Problems-with-VC-2008-Express,['gang I\'m trying to use VC # Express 2008. O...,['birkir.gunnarsson' 'lras' 'birkir.gunnarsson...,['https://www.freelists.org/post/program-l/Pro...,['2009-05-19T19:12:22.000000000' '2009-05-19T1...
4,1117,VB6-Tab-Order,"[""Hi all, I have a VB form which shows a numbe...",['david.lant' 'markalong64' 'jamal.mazrui' 'da...,['https://www.freelists.org/post/program-l/VB6...,['2005-07-28T15:12:17.000000000' '2005-07-28T1...


In [50]:
data['account']

0     ['chojiro1990' 'soronel.haetir' 'chojiro1990' ...
1     ['hedvig.jung' 'dzhovani.chemishanov' 'hedvig....
2     ['justind' 'david.lant' 'justind' 'travis' 'ju...
3     ['birkir.gunnarsson' 'lras' 'birkir.gunnarsson...
4     ['david.lant' 'markalong64' 'jamal.mazrui' 'da...
                            ...                        
95                          ['cmusic789' 'james.homme']
96    ['m10fayed' 'abletec' 'ntsiklauri2' 'spg1111' ...
97                 ['jhomme' 'jhomme' 'isidor.nikolic']
98    ['programmer651' 'james.corbett' 'joseph.lee22...
99    ['pmorales' 'rbreiten' 'jude.dashiell' 'jude.d...
Name: account, Length: 100, dtype: object

### 1. Regular Expression (RegEx) (2pt)

#### 1-1. RegEx to extract usernames (1pt)

In [51]:
data['account'].iloc[0]

"['chojiro1990' 'soronel.haetir' 'chojiro1990' 'soronel.haetir'\r\n 'chojiro1990' 'soronel.haetir' 'chojiro1990' 'soronel.haetir'\r\n 'chojiro1990']"

In [52]:
#YOUR CODE HERE
data['extracted_usernames'] = data['account'].apply(lambda x: re.findall(r"'([a-zA-Z0-9._-]+)'", x))
data[['account', 'extracted_usernames']].head()

,account,extracted_usernames
0,['chojiro1990' 'soronel.haetir' 'chojiro1990' ...,"[chojiro1990, soronel.haetir, chojiro1990, sor..."
1,['hedvig.jung' 'dzhovani.chemishanov' 'hedvig....,"[hedvig.jung, dzhovani.chemishanov, hedvig.jung]"
2,['justind' 'david.lant' 'justind' 'travis' 'ju...,"[justind, david.lant, justind, travis, justind..."
3,['birkir.gunnarsson' 'lras' 'birkir.gunnarsson...,"[birkir.gunnarsson, lras, birkir.gunnarsson, j..."
4,['david.lant' 'markalong64' 'jamal.mazrui' 'da...,"[david.lant, markalong64, jamal.mazrui, david...."


#### 1-2. RegEx to extract date (1pt)

In [53]:
data['date'].iloc[0]

"['2013-06-05T09:50:54.000000000' '2013-06-05T15:42:35.000000000'\r\n '2013-06-05T16:28:36.000000000' '2013-06-05T19:52:01.000000000'\r\n '2013-06-06T09:33:53.000000000' '2013-06-06T15:31:31.000000000'\r\n '2013-06-06T19:15:11.000000000' '2013-06-06T20:04:44.000000000'\r\n '2013-06-07T05:05:21.000000000']"

In [54]:
#YOUR CODE HERE
data['extracted_dates'] = data['date'].apply(lambda x: re.findall(r'(\d{4})-(\d{2})-(\d{2})', x))
data[['date', 'extracted_dates']].head()

,date,extracted_dates
0,['2013-06-05T09:50:54.000000000' '2013-06-05T1...,"[(2013, 06, 05), (2013, 06, 05), (2013, 06, 05..."
1,['2021-05-18T14:26:59.000000000' '2021-05-18T1...,"[(2021, 05, 18), (2021, 05, 18), (2021, 05, 18)]"
2,['2005-08-31T08:52:21.000000000' '2005-08-31T0...,"[(2005, 08, 31), (2005, 08, 31), (2005, 08, 31..."
3,['2009-05-19T19:12:22.000000000' '2009-05-19T1...,"[(2009, 05, 19), (2009, 05, 19), (2009, 05, 19..."
4,['2005-07-28T15:12:17.000000000' '2005-07-28T1...,"[(2005, 07, 28), (2005, 07, 28), (2005, 07, 28..."


### 2. Pandas and Text mining (6pt)

#### 2-1. Count the number of users involved in the email conversation (1pt)

In [55]:
#YOUR CODE HERE
data['account'].apply(lambda x: pd.Series(x).nunique()).sort_values(ascending=False)

0     1
63    1
73    1
72    1
71    1
     ..
30    1
29    1
28    1
27    1
99    1
Name: account, Length: 100, dtype: int64

#### 2-2. Lowercase the body column (1pt)

In [56]:
#YOUR CODE HERE
data['body_lower'] = data['body'].apply(lambda x: x.lower())
print(data['body_lower'].iloc[0])

["hi people! right now, i'm currently doing a project college with two others. project is basically a scenario where a company needs a server set up, proper networking"
 'if you want to append to a file rather than overwrite you first need to try file. open ( ) only use file. create ( ) if the open fails ( by throwing a system. io. filenotfoundexception ).'
 "hi! yeah, i'm going to fix that up later on. i'm more concerned with getting the right numbers to log, as those above 25, but nothing else. any number above 25, for example, should be logged. everything else should not be. i've had a look around, but can't find anything consistent."
 'restricting the write to overheating alerts is a matter of understanding the data produced by your probe. you are a far better position to know what data your probe returns than i am. place to make the change would most likely be p _ datareceived would be something like: string databuffer = " "; static void p _ datareceived ( object sender, serialdat

#### 2-3. Removing stopwords (1pt)

In [57]:
#YOUR CODE HERE
stop=stopwords.words('english')
data['stopword']=data['body_lower'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))
data['body_lower'].iloc[98]

'[\'tell me if you guys get an attachment if i send one. recently, i created a application the javascript programming language. program is called a " tape loop ", for i like to compare it to the modern day version of looping through data cells the " tape. " this app produces no output per loop iteration, for it is a " speed test. " in the app, you will see a blank box, a button labeled " start the tape loop, " a box with some words it. let\\\'s say you type 1000 the first box. from the time you hit " start the tape loop " to the time the words the third box change to the number you just typed, it has done 1000 computations. it becomes obvious with numbers like 10000 ( ten thousand ), where it takes a few seconds for the box on the right to be updated. it is a speed test of the operating system, the browser, the javascript interpreter of the browser; the third being mostly what it is. z\'\r\n \'z: " tell me if you guys get an attachment if i send one. ", wouldn\\\'t that question answer

In [58]:
data['stopword'].iloc[98]

'[\'tell guys get attachment send one. recently, created application javascript programming language. program called " tape loop ", like compare modern day version looping data cells " tape. " app produces output per loop iteration, " speed test. " app, see blank box, button labeled " start tape loop, " box words it. let\\\'s say type 1000 first box. time hit " start tape loop " time words third box change number typed, done 1000 computations. becomes obvious numbers like 10000 ( ten thousand ), takes seconds box right updated. speed test operating system, browser, javascript interpreter browser; third mostly is. z\' \'z: " tell guys get attachment send one. ", wouldn\\\'t question answer it\\\'s self sent attachment?\' "hi, get attachments you." \'sorry, mailing list systems delete attachments, could messing up. reply message. z\' \'hi: jumps, permitted send attachments list.\']'

#### 2-4. Tokenization and removing punctuations (1pt)

In [59]:
#YOUR CODE HERE
data['token']=data['stopword'].apply(lambda x: word_tokenize(x))
data['token'].iloc[98][:10]

data['punct_token']=data['token'].apply(lambda x: [word for word in x if word.isalnum()])
data['punct_token'].iloc[98][:10]

['guys',
 'get',
 'attachment',
 'send',
 'one',
 'recently',
 'created',
 'application',
 'javascript',
 'programming']

#### 2-5. Lemmatization (1pt)

In [60]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Zain\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [61]:
#YOUR CODE HERE
import pandas as pd
from nltk.stem import WordNetLemmatizer

wn = WordNetLemmatizer()
def lemmatization(token_txt):
    text = [wn.lemmatize(word) for word in token_txt]
    return text
data['body_lemmatization'] = data['body_lower'].apply(lambda x: lemmatization(x))
data.head()

,thread_id,thread_name,body,account,url,date,extracted_usernames,extracted_dates,body_lower,stopword,token,punct_token,body_lemmatization
0,4007,Temperature-monitoring-project-in-C,"[""Hi people! Right now, I'm currently doing a ...",['chojiro1990' 'soronel.haetir' 'chojiro1990' ...,['https://www.freelists.org/post/program-l/Tem...,['2013-06-05T09:50:54.000000000' '2013-06-05T1...,"[chojiro1990, soronel.haetir, chojiro1990, sor...","[(2013, 06, 05), (2013, 06, 05), (2013, 06, 05...","[""hi people! right now, i'm currently doing a ...","[""hi people! right now, i'm currently project ...","[[, ``, hi, people, !, right, now, ,, i, 'm, c...","[hi, people, right, now, i, currently, project...","[[, "", h, i, , p, e, o, p, l, e, !, , r, i, ..."
1,7725,subscription,"['I am. I write you from Budapest, Hungary. I ...",['hedvig.jung' 'dzhovani.chemishanov' 'hedvig....,['https://www.freelists.org/post/program-l/sub...,['2021-05-18T14:26:59.000000000' '2021-05-18T1...,"[hedvig.jung, dzhovani.chemishanov, hedvig.jung]","[(2021, 05, 18), (2021, 05, 18), (2021, 05, 18)]","['i am. i write you from budapest, hungary. i ...","['i am. write budapest, hungary. work area vis...","[[, ', i, am, ., write, budapest, ,, hungary, ...","[i, am, write, budapest, hungary, work, area, ...","[[, ', i, , a, m, ., , i, , w, r, i, t, e, ..."
2,1162,jaws-remote-desktop,"[""Hi All. I'm possibly needing to use windows ...",['justind' 'david.lant' 'justind' 'travis' 'ju...,['https://www.freelists.org/post/program-l/jaw...,['2005-08-31T08:52:21.000000000' '2005-08-31T0...,"[justind, david.lant, justind, travis, justind...","[(2005, 08, 31), (2005, 08, 31), (2005, 08, 31...","[""hi all. i'm possibly needing to use windows ...","[""hi all. i'm possibly needing use windows xp ...","[[, ``, hi, all, ., i, 'm, possibly, needing, ...","[hi, all, i, possibly, needing, use, windows, ...","[[, "", h, i, , a, l, l, ., , i, ', m, , p, ..."
3,2300,Problems-with-VC-2008-Express,['gang I\'m trying to use VC # Express 2008. O...,['birkir.gunnarsson' 'lras' 'birkir.gunnarsson...,['https://www.freelists.org/post/program-l/Pro...,['2009-05-19T19:12:22.000000000' '2009-05-19T1...,"[birkir.gunnarsson, lras, birkir.gunnarsson, j...","[(2009, 05, 19), (2009, 05, 19), (2009, 05, 19...",['gang i\'m trying to use vc # express 2008. o...,['gang i\'m trying use vc # express 2008. firs...,"[[, 'gang, i\, 'm, trying, use, vc, #, express...","[trying, use, vc, express, first, page, open, ...","[[, ', g, a, n, g, , i, \, ', m, , t, r, y, ..."
4,1117,VB6-Tab-Order,"[""Hi all, I have a VB form which shows a numbe...",['david.lant' 'markalong64' 'jamal.mazrui' 'da...,['https://www.freelists.org/post/program-l/VB6...,['2005-07-28T15:12:17.000000000' '2005-07-28T1...,"[david.lant, markalong64, jamal.mazrui, david....","[(2005, 07, 28), (2005, 07, 28), (2005, 07, 28...","[""hi all, i have a vb form which shows a numbe...","[""hi all, vb form shows number rows controls. ...","[[, ``, hi, all, ,, vb, form, shows, number, r...","[hi, all, vb, form, shows, number, rows, contr...","[[, "", h, i, , a, l, l, ,, , i, , h, a, v, ..."


In [62]:
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'): #ADJECTIVE
        return wordnet.ADJ
    elif nltk_tag.startswith('V'): #VERN
        return wordnet.VERB
    elif nltk_tag.startswith('N'): #NOUN        
        return wordnet.NOUN
    elif nltk_tag.startswith('R'): #ADVERB
        return wordnet.ADV
    else:          
        return None

In [63]:
def lemmatize_sentence(sentence):
    # Tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    # Tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged) 
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            # If no tag was found, then use the word as is
            lemmatized_sentence.append(word)
        else:        
            # Else use the tag to lemmatize the word
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

#### 2-6. Count the frequency of unigrams (1pt)

In [64]:
#YOUR CODE HERE
def simple_tokenize(text):
    return text.lower().split()

thread_sample_path = 'thread_sample.csv'
thread_sample_df = pd.read_csv(thread_sample_path)

unigrams_simple = thread_sample_df['body'].astype(str).apply(lambda x: simple_tokenize(x))
unigrams_flat_list_simple = [item for sublist in unigrams_simple for item in sublist]

unigram_counts_simple = Counter(unigrams_flat_list_simple)

unigram_counts_simple.most_common(10)

[('the', 2382),
 ('to', 1856),
 ('i', 1433),
 ('a', 1172),
 ('it', 804),
 ('is', 802),
 ('that', 751),
 ('of', 744),
 ('you', 741),
 ('for', 650)]

### 3. Building edges (2pt)

#### 3-1. Combination

In [65]:
#YOUR CODE HERE
import itertools
edges=[]
for idx, val in data['account'].items():
    edges.extend(list(itertools.combinations(val, 2)))

In [66]:
edges[:5]

[('[', "'"), ('[', 'c'), ('[', 'h'), ('[', 'o'), ('[', 'j')]

#### 3-2. Removing self-loop (1pt)

In [67]:
#YOUR CODE HERE
edges_loop = [edge for edge in edges if edge[0] != edge[1]]

In [68]:
edges_loop[:5]

[('[', "'"), ('[', 'c'), ('[', 'h'), ('[', 'o'), ('[', 'j')]